In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zhon.hanzi import punctuation # 中文标点
import jieba
import re
from bert_serving.client import BertClient
from sklearn.feature_extraction.text import TfidfVectorizer # 创建TF-IDF向量化器
from cnsenti import Sentiment

——————————————————————————————————————  
### 分词

In [3]:
text_1820_path ='F:\\zzqaq\\data\\data_clean_2018_2020.csv'
text_1820 = pd.read_csv(text_1820_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
text_1820["board"] = [str(i).replace("\t","") for i in text_1820["board"]]
text_1820.head(2)

,code,year,text,board
0,000004,2018,讨论与分析一概述年度，公司实现主营业务收入万元，比上年同期增加，实现归属于上市公司股东的净利...,00
1,000004,2020,讨论与分析一概述年是公司业务转型后的第一年，年，公司筹划发行股份收购智游网安股权的重大资产重...,00


In [211]:
# 数每个文本句子个数
sentence_count=[]
for i in range(text_1820.shape[0]):
    text_split_each = re.split(r'(?:[。?!])', text_1820['text'][i])
#     print(i,len(text_split_each))
    sentence_count.append(len(text_split_each))
# 作为特征放入df
text_1820['sentence_count'] = sentence_count
all_text_data['word_count']=word_count
all_text_data['readability']=all_text_data['word_count']/all_text_data['sentence_count']

In [216]:
# 分词生成语料
text_split = list(text_1820['text'])
# 每个句子进行分词
corpus = []
i = 0
for text_to_cut in text_split:
# # 词语生成器
#     print(text_to_cut)
    text_to_cut = re.sub(r'[%s]+' %punctuation,"", text_to_cut) #去标点    
    text_to_cut = re.sub('[a-zA-Z]','',text_to_cut) # 去英文
    text_cut = jieba.cut(text_to_cut, cut_all=False)
    text_cut  = " ".join(text_cut)
    corpus.append(text_cut)
    i+=1
    if(i%1000==0):print(i)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\asus\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built successfully.


1000
2000
3000
4000
5000
6000
7000


In [242]:
# 查看
text_1820.text = corpus
# text_1820.head(2)

# 去停用词
path_stpwrd = 'F:\\zzqaq\\text_data_processs\\哈工大停用词表.txt'
with open(path_stpwrd, 'rb') as fp:
    stopword = fp.read().decode('utf-8')  # 提用词提取
stpwrdlst = stopword.splitlines()  #将停用词表转换为list  
extra_stpwrd=['巨潮','资讯网','√','□','公司','年度报告']
stpwrdlst_ = stpwrdlst + extra_stpwrd

new_corpus = []

for string in corpus:
    words = [word for word in string.split() if word not in stpwrdlst_]
    new_string = ' '.join(words)
    new_corpus.append(new_string)

# 查看
text_1820['text'] = new_corpus
text_1820=text_1820.rename(columns={'text':'corpus'})
text_1820.head(3)

,code,year,corpus,board,sentence_count
0,000004,2018,讨论 分析 概述 年度 实现 主营业务 收入 万元 上年 同期 增加 实现 归属于 上市公司...,00,52
1,000004,2020,讨论 分析 概述 年 业务 转型 后 第一年 年 筹划 发行 股份 收购 智游 网安 股权 ...,00,126
2,000005,2018,讨论 分析 概述 报告 期内 本司 主营业务 架构 调整 如下 交通 清洁 能源 水资源 基...,00,122


In [ ]:
# 保存
text_1820.to_csv('F:\\zzqaq\\data\\corpus_2018_2020.csv',encoding="gb18030",index = False)

In [ ]:
# 读取
corpus_18_20_path ='F:\\zzqaq\\data\\corpus_2018_2020.csv'
corpus_1820 = pd.read_csv(corpus_18_20_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})